<a href="https://colab.research.google.com/github/miladalipour99/Waskom_CurrBiol_2018/blob/master/statistic_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model summaries and statistical tests

In [1]:
import json
import itertools
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.formula.api import logit

In [13]:
def load_fit_results(model, subject=None):
    """Read a json file with cached fit results from a known location."""
    fpath = f"/content/Waskom_CurrBiol_2018/fits/{model}.json" if subject is None else f"/content/Waskom_CurrBiol_2018/fits/{model}_{subject}.json"
    with open(fpath, "r") as fid:
        return json.load(fid)


def get_param_and_ci(res, key):
    """Extract parameter value and compute CI from bootstrap distribution."""
    param = res["params"][key]
    boots = np.fromiter((boot["params"][key] for boot in res["bootstraps"]), np.float)
    ci = np.percentile(boots, [2.5, 97.5])
    return param, ci

In [3]:
!git clone https://github.com/kianilab/Waskom_CurrBiol_2018

Cloning into 'Waskom_CurrBiol_2018'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 55 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (55/55), done.


In [4]:
trial_data = pd.read_csv("/content/Waskom_CurrBiol_2018/data/trial_data.csv")
pulse_data = pd.read_csv("/content/Waskom_CurrBiol_2018/data/pulse_data.csv")
subjects = ["S1", "S2", "S3", "S4", "S5"]
models = ["linear", "extrema", "counting", "leaky"]
trial_grouper = ["subject", "timing", "session", "run", "trial"]

In [5]:
pd.set_option("display.precision", 3)

---

## Basic dataset statistics

In [6]:
print(f"{len(trial_data):,d} Total trials")

14,869 Total trials


In [7]:
shorter_trials = trial_data.query("timing == 'shorter'")
print(f"{len(shorter_trials):,d} shorter trials")
longer_trials = trial_data.query("timing == 'longer'")
print(f"{len(longer_trials):,d} longer trials")

6,500 shorter trials
8,369 longer trials


In [8]:
for subj, subj_data in trial_data.groupby("subject"):
    print(f"{subj}: {len(subj_data):,d} trials")

S1: 3,059 trials
S2: 2,882 trials
S3: 2,913 trials
S4: 2,987 trials
S5: 3,028 trials


In [9]:
timing = trial_data.trial_dur.describe()
print(f"""
Trial duration:
  mean+/-std: {timing['mean']:.1f}+/-{timing['std']:.2f} s
  range: {timing['min']:.1f}–{timing['max']:.1f} s
""")


Trial duration:
  mean+/-std: 10.1+/-5.59 s
  range: 2.2–34.0 s



---

## Model parameters

In [14]:
params = {}
for model, subj in itertools.product(models, subjects):
    fit = load_fit_results(model, subj)
    for param in fit["params"]:
        val, ci = get_param_and_ci(fit, param)
        params[(model, param, subj)] = [val] + list(ci)
params = pd.DataFrame(params, index=["MLE", "CI (low)", "CI (high)"])

<ipython-input-13-7f1bfaaf8eb2>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  boots = np.fromiter((boot["params"][key] for boot in res["bootstraps"]), np.float)


In [15]:
params.astype(object).T

MLE CI (low) CI (high)
linear   sigma     S1  0.424    0.395     0.453
                   S2  0.457    0.424     0.489
                   S3  0.476    0.442      0.51
                   S4  0.568    0.523     0.615
                   S5  0.412    0.383     0.438
extrema  sigma     S1  0.395    0.363     0.426
         theta     S1  0.487    0.451     0.521
         sigma     S2  0.428    0.395      0.46
         theta     S2  0.503    0.466      0.54
         sigma     S3  0.421    0.383     0.458
         theta     S3  0.503    0.466     0.541
         sigma     S4  0.497    0.457     0.535
         theta     S4  0.513     0.47     0.556
         sigma     S5  0.406    0.374     0.436
         theta     S5  0.527    0.493     0.559
counting sigma     S1  0.323    0.298     0.347
                   S2  0.348     0.32     0.377
                   S3  0.379    0.349     0.411
                   S4  0.447    0.412     0.484
                   S5  0.324    0.298     0.348
leaky    sigma_eta S1  0.424    0.394     0.453
         sigma_eps S1    0.0      0.0       0.0
         alpha     S1    0.0      0.0     0.009
         sigma_eta S2  0.457    0.423     0.491
         sigma_eps S2    0.0      0.0       0.0
         alpha     S2    0.0      0.0     0.026
         sigma_eta S3  0.476    0.436     0.508
         sigma_eps S3    0.0      0.0      0.08
         alpha     S3    0.0      0.0     0.011
         sigma_eta S4  0.568    0.465     0.605
         sigma_eps S4    0.0      0.0      0.16
         alpha     S4    0.0      0.0       0.0
         sigma_eta S5  0.408    0.374     0.434
         sigma_eps S5    0.0      0.0     0.049
         alpha     S5  0.039    0.008      0.07

---

## Log-likelihoods and model comparisons

In [16]:
loglikes = {}
for model in models:
    agg, agg_cv = 0, 0
    for subj in subjects:
        fit = load_fit_results(model, subj)
        loglikes[(model, subj)] = [fit["loglike"], fit["cv_loglike"]]
        agg += fit["loglike"]
        agg_cv += fit["cv_loglike"]
    loglikes[(model, "agg")] = [agg, agg_cv]
loglikes = pd.DataFrame(loglikes, index=["loglike", "cv_loglike"])

In [17]:
loglikes[models].T

loglike  cv_loglike
linear   S1   -925.781    -927.538
         S2   -920.391    -922.830
         S3   -954.744    -955.920
         S4  -1123.667   -1125.917
         S5   -880.606    -882.946
         agg -4805.189   -4815.151
extrema  S1  -1099.216   -1101.284
         S2  -1077.112   -1079.987
         S3  -1085.826   -1089.015
         S4  -1218.818   -1222.534
         S5  -1108.981   -1111.011
         agg -5589.953   -5603.831
counting S1   -940.396    -942.150
         S2   -919.868    -922.141
         S3   -977.324    -979.239
         S4  -1129.088   -1132.006
         S5   -908.208    -911.157
         agg -4874.884   -4886.694
leaky    S1   -925.781    -927.538
         S2   -920.391    -922.954
         S3   -954.744    -955.920
         S4  -1123.667   -1125.928
         S5   -875.545    -881.457
         agg -4800.128   -4813.797

### Linear Integration vs. Extrema Detection

In [18]:
loglikes["linear"] - loglikes["extrema"]

,S1,S2,S3,S4,S5,agg
loglike,173.435,156.721,131.081,95.151,228.375,784.764
cv_loglike,173.746,157.158,133.095,96.617,228.065,788.680


### Linear Integration vs. Counting

In [19]:
loglikes["linear"] - loglikes["counting"]

,S1,S2,S3,S4,S5,agg
loglike,14.615,-0.523,22.579,5.421,27.602,69.695
cv_loglike,14.611,-0.688,23.320,6.089,28.211,71.543


### Linear Integration vs. Leaky Integration

In [20]:
loglikes["linear"] - loglikes["leaky"]

,S1,S2,S3,S4,S5,agg
loglike,-1.003e-07,-1.027e-07,-7.947e-08,-2.936e-07,-5.061,-5.061
cv_loglike,-2.156e-04,1.244e-01,1.519e-04,1.093e-02,-1.489,-1.354


In [21]:
def likeratio_test(test, null, dof, cv=False):

    col = "cv_loglike" if cv else "loglike"
    stat = 2 * (loglikes.loc[col, test] - loglikes.loc[col, null])
    pval = stats.chi2(dof).sf(stat)
    return pd.DataFrame(np.c_[stat, pval],
                        columns=["stat", "pval"],
                        index=loglikes.loc["loglike", test].index)

In [22]:
likeratio_test("leaky", "linear", dof=2).round(3).T

,S1,S2,S3,S4,S5,agg
stat,0.0,0.0,0.0,0.0,10.122,10.122
pval,1.0,1.0,1.0,1.0,0.006,0.006


In [23]:
likeratio_test("leaky", "linear", dof=2, cv=True).round(3).T

,S1,S2,S3,S4,S5,agg
stat,0.0,-0.249,-0.0,-0.022,2.978,2.708
pval,1.0,1.000,1.0,1.000,0.226,0.258


---

## Statistical test of accuracy improvements

In [24]:
def test_odd_vs_even(trial_data):
    """Use logistic regression to test """
    trials = trial_data.pulse_count < 5
    tsub = trial_data.loc[trials, ["pulse_count", "correct"]]
    y = tsub["correct"]
    df = pd.DataFrame(dict(
        correct=tsub["correct"],
        parity=tsub["pulse_count"] % 2 == 0,
        magnitude=tsub["pulse_count"] > 2,
    )).astype(int)
    m = logit("correct ~ magnitude + parity", df).fit(disp=False)
    return m

In [25]:
m = test_odd_vs_even(trial_data)
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                correct   No. Observations:                13427
Model:                          Logit   Df Residuals:                    13424
Method:                           MLE   Df Model:                            2
Date:                Mon, 19 Dec 2022   Pseudo R-squ.:                0.009477
Time:                        23:35:16   Log-Likelihood:                -7198.2
converged:                       True   LL-Null:                       -7267.1
Covariance Type:            nonrobust   LLR p-value:                 1.232e-30
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9589      0.029     32.867      0.000       0.902       1.016
magnitude      0.4483      0.046      9.842      0.000       0.359       0.538
parity         0.2517      0.042      5.949      0.000       0.169       0.335
==============================================================================
"""

In [26]:
print(f"P = {m.pvalues.parity:.2g}")

P = 2.7e-09


In [27]:
for subj in subjects:
    tdata = trial_data.query("subject == @subj")
    m = test_odd_vs_even(tdata)
    print(f"{subj}: P = {m.pvalues.parity / 2:.2g}")

S1: P = 0.0023
S2: P = 0.0069
S3: P = 0.0088
S4: P = 0.0031
S5: P = 0.0011


----

## Statistical test of differences between shorter and longer gaps

In [28]:
def test_longer_vs_shorter(pulse_data):
    pulse_data = pulse_data.merge(trial_data, on=trial_grouper)
    fit_data = pulse_data.groupby(trial_grouper).mean().reset_index()
    m = logit("response ~ pulse_llr * timing", fit_data).fit(disp=False)
    return m

In [29]:
m = test_longer_vs_shorter(pulse_data)
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               response   No. Observations:                14869
Model:                          Logit   Df Residuals:                    14865
Method:                           MLE   Df Model:                            3
Date:                Mon, 19 Dec 2022   Pseudo R-squ.:                  0.5393
Time:                        23:35:56   Log-Likelihood:                -4747.5
converged:                       True   LL-Null:                       -10304.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       0.1367      0.034      3.981      0.000       0.069       0.204
timing[T.shorter]              -0.1552      0.053     -2.950      0.003      -0.258      -0.052
pulse_llr                       5.4972      0.118     46.579      0.000       5.266       5.728
pulse_llr:timing[T.shorter]     0.2910      0.184      1.582      0.114      -0.070       0.652
===============================================================================================
"""

In [30]:
for subj in subjects:
    pdata = pulse_data.query("subject == @subj")
    m = test_longer_vs_shorter(pdata)
    print(f"{subj}: P = {m.pvalues['pulse_llr:timing[T.shorter]']:.2g}")

S1: P = 0.16
S2: P = 0.91
S3: P = 0.42
S4: P = 0.03
S5: P = 0.52
